# Fama–French Three-Factor Model (3F)
This notebook estimates the Fama–French Three-Factor Model for a set of assets by regressing excess asset returns on the Market, SMB, and HML factors.

**Source:** Essentials of Financial Economics  
**Authors:** Michael Donadelli, Michele Costola, Ivan Gufler  
**Date:** May 8, 2025


## 0. Preliminaries

This Python notebook estimates a Fama-French three factor model using:
- Market risk
- Size factor 
- Value factor

We use returns of the 5 oldest stocks in the DJIA index: Procter and Gamble, 3M, Merkx, IBM and American Express, from February 2000 to December 2022. The risk-free rate is from Fama-French library and proxied by the one-month US Treasury bill. 


## 1. Import Libraries


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


## 2. Load and Prepare Data


In [5]:
Ret = pd.read_excel('../Data/Returns.xlsx')
Factors = pd.read_excel('../Data/FF_Factors.xlsx')

FF = np.array(Factors.iloc[1:, 1:4]) / 100
Rf = np.array(Factors.iloc[1:, 4]) / 100
ExRet = np.array(Ret.iloc[:, 1:]) - Rf.reshape(-1, 1)



## 3. Run Three-Factor Regressions
We regress each asset's excess returns on the three Fama–French factors:

$$
R_{i,t} - R_f = \alpha_i + \beta_{mkt} (Mkt-RF)_t + \beta_{smb} SMB_t + \beta_{hml} HML_t + \varepsilon_t.
$$


In [6]:
nS, mS = ExRet.shape

alpha = np.empty((2, mS))
beta_mkt = np.empty((2, mS))
beta_smb = np.empty((2, mS))
beta_hml = np.empty((2, mS))
R2 = np.empty((1, mS))

for i in range(mS):
    X = sm.add_constant(FF)
    y = ExRet[:, i]
    model = sm.OLS(y, X)
    results = model.fit()

    alpha[:, i] = [round(results.params[0], 5), round(results.tvalues[0], 5)]
    beta_mkt[:, i] = [round(results.params[1], 5), round(results.tvalues[1], 5)]
    beta_smb[:, i] = [round(results.params[2], 5), round(results.tvalues[2], 5)]
    beta_hml[:, i] = [round(results.params[3], 5), round(results.tvalues[3], 5)]
    R2[0, i] = round(results.rsquared_adj, 5)


## 4. Display Results


In [7]:
Stocks = pd.DataFrame(
    data=np.concatenate((alpha, beta_mkt, beta_smb, beta_hml, R2)),
    columns=Ret.columns[1:],
    index=[
        'alpha', '(alpha t-stat)',
        'beta_mkt', '(beta_mkt t-stat)',
        'beta_smb', '(beta_smb t-stat)',
        'beta_hml', '(beta_hml t-stat)',
        'Adj. R2'
    ]
)

print('Fama–French 3-Factor Model Results:')
display(Stocks)



Fama–French 3-Factor Model Results:


,PG,MMM,IBM,MRK,AXP
alpha,0.00468,0.00140,0.00007,0.00398,-0.00064
(alpha t-stat),1.56019,0.49451,0.01851,1.07701,-0.16291
beta_mkt,0.30705,0.80851,0.98619,0.60997,1.33952
(beta_mkt t-stat),4.60308,12.86181,12.56610,7.41749,15.24874
beta_smb,-0.19219,-0.06221,-0.26803,-0.85873,-0.20480
(beta_smb t-stat),-1.94879,-0.66937,-2.31002,-7.06310,-1.57692
beta_hml,0.16694,0.25193,-0.14787,-0.00555,0.70599
(beta_hml t-stat),1.91695,3.06969,-1.44318,-0.05173,6.15578
Adj. R2,0.07991,0.39661,0.36562,0.22808,0.50187


## 5. Save Results


In [11]:
Stocks.to_csv('CAPM_3F_Stock.csv', index_label='Metric')
print("Results saved to 'CAPM_3F_Stock.csv'.")


Results saved to 'CAPM_3F_Stock.csv'.


## 6. Interpretation
- **Alpha (α)**: abnormal return unexplained by the three factors.
- **Market Beta**: sensitivity to overall market risk.
- **SMB Beta**: exposure to size (small–big) returns.
- **HML Beta**: exposure to value (high–low book-to-market) returns.
- **Adjusted R²**: model explanatory power.
